## Demonstration of coordinate transformations between orbital elements and Cartesian coordinates


The goal here is to demonstrate how these coordinate transformations work, and what they do. The goal is to from a set of orbital elements in the usual Keplerian basis $(a, e, i, \Omega, \omega, \mathcal{M})$ or the cometary basis $(q, e, i, \Omega, \omega, T_p)$ into a set of state vectors $(x, y, z, v_x, v_y, v_z)$. 

Under the hood, the transformations are using the universal variable formulation of the two body problem, so that eccentric ($0 \leq e <1$), parabolic ($e=1$) and hyperbolic ($e>1$) orbits can be handled correclty and simultaneously. 


For the reader interested into getting in the details, [Dandy's Fundamental of Celestial Mechanics](https://www.google.com/books/edition/Fundamentals_of_Celestial_Mechanics/N5h-AAAAIAAJ) is a good reference. The implementation inside `sorcha` follows [Everhart & Pitkin (1983)](https://ui.adsabs.harvard.edu/abs/1983AmJPh..51..712E/abstract).



### A simple example
Let's start with the simplest possible example. Let's define the reduced mass of the system to $\mu = 1$, and the reference time for the elements (i.e. the epoch where they are defined) to $t = 0$, and unitless distances and times. 

Let's start with a simple series of orbits:
- $q = 10$, $e = 0$, $i = 0^\circ$, $\Omega = 10^\circ$, $\omega = 0^\circ$, $T_p = 0$
- $q = 10$, $e = 0.1$, $i = 0^\circ$, $\Omega = 10^\circ$, $\omega = 0^\circ$, $T_p = 0$
- $q = 10$, $e = 0.9999$, $i = 0^\circ$, $\Omega = 10^\circ$, $\omega = 0^\circ$, $T_p = 0$
- $q = 10$, $e = 1$, $i = 0^\circ$, $\Omega = 10^\circ$, $\omega = 0^\circ$, $T_p = 0$
- $q = 10$, $e = 1.0001$, $i = 0^\circ$, $\Omega = 10^\circ$, $\omega = 0^\circ$, $T_p = 0$
- $q = 10$, $e = 6.$, $i = 0^\circ$, $\Omega = 10^\circ$, $\omega = 0^\circ$, $T_p = 0$

(In all of these cases, the object is at perihelion at the reference time)

In [ ]:
import numpy as np 
from sorcha.ephemeris.orbit_conversion_utilities import universal_cartesian, universal_keplerian

In [ ]:
# define orbits (no e)
q = 10
i = 0 * np.pi/180
Omega = 10 * np.pi/180
omega = 0
Tp = 0
#define constants
epochMJD_TDB = 0
mu = 1

In [ ]:
for e in [0, 0.1, 0.9999, 1.0, 1.0001, 6.]:
    
	x,y,z,vx,vy,vz = universal_cartesian(mu, q, e, i, Omega, omega, Tp, epochMJD_TDB)
	print(f'{e}: {x}, {y}, {z}, {vx}, {vy}, {vz}')

Unsurprisingly (in other words, intentionally), all orbits are at the same $(x,y,z)$ position, just different velocities (note $v_z = 0$ in all cases because $i = 0^\circ$).

We can also invert these transformations:

In [ ]:
for e in [0, 0.1, 0.9999, 1.0, 1.0001, 6.]:
    
	x,y,z,vx,vy,vz = universal_cartesian(mu, q, e, i, Omega, omega, Tp, epochMJD_TDB)
	q_out,e_out,i_out,Omega_out,omega_out,Tp_out = universal_keplerian(mu, x, y, z, vx, vy, vz, epochMJD_TDB)
	print(f'{e}: {q_out}, {e_out}, {i_out}, {Omega_out}, {omega_out}, {Tp_out}')

Note that any differences are of order $10^{-14}$ or less - this only slightly above the machine precision level for doubles - ensuring higher precision would sacrifice runtimes and convergence for the solutions. For the all applications of `sorcha`, this is more than enough.

### Real world example #1

Let's also go through these transformations for a real asteroid. Let's take asteroid 3666 Holman. Using JPL Horizons, we have that:

```
  2457545.5000000000 = A.D. 2016-Jun-06 00:00:00.0000 TDB     
   EC= .1273098034941495   QR= 2.719440725689577   TP= 2457934.5526586706      
   OM= 120.3869311657135   W=  55.06308036878693   IN= 2.363582123711951       
```
At the same epoch, 
```
 X =-7.569545429706993E-02 Y = 3.024083648650882E+00 Z =-6.044399403284755E-02
 VX=-9.914117209213893E-03 VY=-1.485136186100886E-03 VZ= 3.840061650310168E-04
```
in these units (distances are in au and times are in days), $\mu = 2.9591220828559115 \cdot 10^{-4} \, \mathrm{au}^3/\mathrm{d}^2$. 

Let's copy these numbers directly:


In [ ]:
EC= .1273098034941495 
QR= 2.719440725689577  
TP= 2457934.5526586706      
OM= 120.3869311657135 * np.pi/180
W=  55.06308036878693   * np.pi/180 
IN= 2.363582123711951      * np.pi/180
epochMJD_TDB = 2457545.5

In [ ]:
mu = 2.9591220828559115e-04
x, y, z, vx, vy, vz = universal_cartesian(mu, QR, EC, IN, OM, W, TP, epochMJD_TDB)
print(x,y,z,vx,vy,vz)

In [ ]:
 X =-7.569545429706993E-02 
 Y = 3.024083648650882E+00 
 Z =-6.044399403284755E-02
 VX=-9.914117209213893E-03 
 VY=-1.485136186100886E-03
 VZ= 3.840061650310168E-04

print(x- X, y- Y, z - Z, vx - VX, vy - VY, vz - VZ)

The differences are within reasonable tolerance limits.

So finally, if we want to rotate these from the ecliptic coordinate frame to the equatorial frame, we have that:

In [ ]:
from sorcha.ephemeris.simulation_constants import ECL_TO_EQ_ROTATION_MATRIX

In [ ]:
pos_ecl = np.array([x,y,z])
vel_ecl = np.array([vx,vy,vz])

pos_eq = ECL_TO_EQ_ROTATION_MATRIX @ pos_ecl
vel_eq = ECL_TO_EQ_ROTATION_MATRIX @ vel_ecl

print(pos_eq, vel_eq)

### Real world example #2 

As a second test, let's compare against 2I/Borisov, doing the inverse transformations:

From JPL, again:

```
2460188.500000000 = A.D. 2023-Sep-01 00:00:00.0000 TDB 
 EC= 3.345550605771202E+00 QR= 1.998821321255714E+00 IN= 4.412305585291305E+01
 OM= 3.080238378883400E+02 W = 2.091328388499988E+02 Tp=  2458825.978725019377
```
```
 X =-6.658710421827730E-01 Y =-2.313385391700901E+01 Z =-1.432926077530441E+01
 VX= 1.089064793811738E-03 VY=-1.681872446865637E-02 VZ=-9.215726774314793E-03
```


In [ ]:
X =-6.658710421827730E-01 
Y =-2.313385391700901E+01 
Z =-1.432926077530441E+01
VX= 1.089064793811738E-03 
VY=-1.681872446865637E-02 
VZ=-9.215726774314793E-03

EC= 3.345550605771202E+00 
QR= 1.998821321255714E+00 
IN= 4.412305585291305E+01 * np.pi/180
OM= 3.080238378883400E+02 *np.pi/180
W = 2.091328388499988E+02  * np.pi/180
Tp=  2458825.978725019377


epochMJD_TDB = 2460188.500000000

q, e, incl, longnode, argperi, tp = universal_keplerian(mu, X, Y, Z, VX, VY, VZ, epochMJD_TDB)

 
print(q, e, incl, longnode, argperi, Tp)
print(q-QR, e-EC, incl-IN, longnode-OM, argperi-W, tp - Tp)


Note that the differences in $\Omega$ and $\omega$ are factors of $2 \pi$ coming from choices about the ranges of these angles:

In [ ]:
print(longnode-OM + 2 * np.pi, argperi-W + 2 * np.pi)

### But what about the mean anomaly?


So far, we have used $(q, e, i, \Omega, \omega, T_p)$ instead of $(a, e, i, \Omega, \omega, \mathcal{M})$. For a bound orbit, the second representation is also possible (and oftentimes simpler/easier to interpret). In these cases, the usual conversions apply:
$$ q = a(1-e)$$ 
$$ T_p = t_0 - \mathcal{M} \sqrt{a^3/\mu} $$

Going back to our first example:

In [ ]:
a = 10
e = 0.1
i = 0 * np.pi/180
Omega = 10 * np.pi/180
omega = 0
M = 0 * np.pi/180 #note that we need radians here as well! 
#define constants
epochMJD_TDB = 0
mu = 1

print(universal_cartesian(mu, a * (1-e), e, i, Omega, omega, epochMJD_TDB - M * np.sqrt(a**3/mu), epochMJD_TDB))

The orbit is still $q = a(1-e) = 9$ units from the center at perihelion ($\mathcal{M} = 0^\circ$), as desired.